## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_to_load = "C://Users/pampe/Desktop/WeatherlyPy/WeatherPy_Database.csv"

city_data_df = pd.read_csv(city_data_to_load )
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Upernavik,GL,72.7868,-56.1549,27.03,56,37,3.04,scattered clouds
1,1,Vaini,TO,-21.2000,-175.2000,62.76,100,20,0.00,few clouds
2,2,Novobirilyussy,RU,56.9514,90.6661,33.04,97,88,4.25,overcast clouds
3,3,Port Elizabeth,ZA,-33.9180,25.5701,63.10,82,90,26.46,overcast clouds
4,4,Magnolia,US,33.2671,-93.2393,83.46,50,21,5.64,few clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Magnolia,US,33.2671,-93.2393,83.46,50,21,5.64,few clouds
9,9,Victoria,HK,22.2855,114.1577,84.22,85,0,1.99,light rain
10,10,Saint George,US,37.1041,-113.5841,83.53,42,0,1.01,clear sky
15,15,Georgetown,MY,5.4112,100.3354,78.89,94,40,10.00,light rain
19,19,Kavieng,PG,-2.5744,150.7967,81.52,78,52,8.39,light rain
20,20,Puerto Ayora,EC,-0.7393,-90.3518,76.96,84,99,9.57,overcast clouds
23,23,Barretos,BR,-20.5572,-48.5678,94.57,24,62,1.36,broken clouds
34,34,Suluq,LY,31.6682,20.2521,76.87,53,25,14.67,scattered clouds
38,38,Puerto Carreno,CO,6.1890,-67.4859,88.99,59,33,7.14,scattered clouds
39,39,Bubaque,GW,11.2833,-15.8333,84.38,71,95,2.46,overcast clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Magnolia,US,33.2671,-93.2393,83.46,50,21,5.64,few clouds
9,9,Victoria,HK,22.2855,114.1577,84.22,85,0,1.99,light rain
10,10,Saint George,US,37.1041,-113.5841,83.53,42,0,1.01,clear sky
15,15,Georgetown,MY,5.4112,100.3354,78.89,94,40,10.00,light rain
19,19,Kavieng,PG,-2.5744,150.7967,81.52,78,52,8.39,light rain
20,20,Puerto Ayora,EC,-0.7393,-90.3518,76.96,84,99,9.57,overcast clouds
23,23,Barretos,BR,-20.5572,-48.5678,94.57,24,62,1.36,broken clouds
34,34,Suluq,LY,31.6682,20.2521,76.87,53,25,14.67,scattered clouds
38,38,Puerto Carreno,CO,6.1890,-67.4859,88.99,59,33,7.14,scattered clouds
39,39,Bubaque,GW,11.2833,-15.8333,84.38,71,95,2.46,overcast clouds


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Magnolia,US,83.46,few clouds,33.2671,-93.2393,
9,Victoria,HK,84.22,light rain,22.2855,114.1577,
10,Saint George,US,83.53,clear sky,37.1041,-113.5841,
15,Georgetown,MY,78.89,light rain,5.4112,100.3354,
19,Kavieng,PG,81.52,light rain,-2.5744,150.7967,
20,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
23,Barretos,BR,94.57,broken clouds,-20.5572,-48.5678,
34,Suluq,LY,76.87,scattered clouds,31.6682,20.2521,
38,Puerto Carreno,CO,88.99,scattered clouds,6.1890,-67.4859,
39,Bubaque,GW,84.38,overcast clouds,11.2833,-15.8333,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}" 
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:   
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(KeyError, IndexError):
        print("Hotel not found ....skipping.")
hotel_df.head(10)        

Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Magnolia,US,83.46,few clouds,33.2671,-93.2393,Budget Inn
9,Victoria,HK,84.22,light rain,22.2855,114.1577,Mini Hotel Central
10,Saint George,US,83.53,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
15,Georgetown,MY,78.89,light rain,5.4112,100.3354,Cititel Penang
19,Kavieng,PG,81.52,light rain,-2.5744,150.7967,Nusa Island Retreat
20,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
23,Barretos,BR,94.57,broken clouds,-20.5572,-48.5678,Dan Inn Barretos
34,Suluq,LY,76.87,scattered clouds,31.6682,20.2521,منزل عبدالواحد خليفة الفاخري
38,Puerto Carreno,CO,88.99,scattered clouds,6.1890,-67.4859,Casa Hotel Victoria
39,Bubaque,GW,84.38,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!= ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Magnolia,US,83.46,few clouds,33.2671,-93.2393,Budget Inn
9,Victoria,HK,84.22,light rain,22.2855,114.1577,Mini Hotel Central
10,Saint George,US,83.53,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
15,Georgetown,MY,78.89,light rain,5.4112,100.3354,Cititel Penang
19,Kavieng,PG,81.52,light rain,-2.5744,150.7967,Nusa Island Retreat
20,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
23,Barretos,BR,94.57,broken clouds,-20.5572,-48.5678,Dan Inn Barretos
34,Suluq,LY,76.87,scattered clouds,31.6682,20.2521,منزل عبدالواحد خليفة الفاخري
38,Puerto Carreno,CO,88.99,scattered clouds,6.1890,-67.4859,Casa Hotel Victoria
39,Bubaque,GW,84.38,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "C:/Users/pampe/Desktop/WeatherlyPy/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))